# MSDS696 Practicum 2 
## Fall 8W2 2020

### Exploration of Colorado County-Level Voting Patterns 2010 - 2020

#### James D. Reed (jreed011@regis.edu)

<img src="../img/Mountains2.jpg" width="700" height="200" />

## Collect Census Data


In [30]:
import requests
import pandas as pd
import censusdata
from pathlib import Path

from decouple import config
import os
%matplotlib inline

In [2]:
# Get CENSUS_KEY
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:  # We are the main py2exe script, not a module
    import sys
    BASE_DIR = os.path.dirname(os.path.abspath(sys.argv[0]))
   
CENSUS_KEY = config('CENSUS_KEY')
GOOGLE_KEY = config('GOOGLE_KEY')
#print(f'CENSUS: {CENSUS_KEY}, GOOGLE: {GOOGLE_KEY}')

---
## Decennial Census of Population

 - Demographics (age, sex, race, family structure))
 - Housing Occupancy and Ownership (vacant./occupied, rent/own)
 - Group Quarters Population (prisons, college dorms)

### Build Population DataFrame

In [24]:
def get_population_df(year = 2010):
    '''
    Name
    P001001 Total Population
    P002001 Total Urban and Rural
    P002002 Total Urban
    P002003 Total Urban inside urbanized areas
    P002004 Total Urban inside urban clusters
    P002005 Total Rural
    P003001 Total Race
    P003002 Total White Alone
    P003003 Total Black or African American alone
    P003004 Total American Indian and Alaska Native alone
    P003005 Total Asian alone
    P003006 Total Native Hawaiian and Other Pacific Islander
    P003007 Total Some Other Race Alone
    P003008 Total Two or More Races
    P004001 Total Hispanic or Latino Origin
    P004002 Total Not Hispanic or Latino Origin
    P004003 Total Hispanic or Latino Origin
    '''
    pop_col_names = ['county_state', 'total_pop', 'total_urb_rur', 'urban', 'urban_ins_ars', 
                     'urban_ins_cls', 'rural',  'total_race', 'white', 'black', 'american_indian',    
                     'asian', 'nat_hawaiian', 'some_othr_race', 'two_or_more_races','hisp_latino',
                     'not_hisp_latino', 'total_hisp_latino', 'state_id', 'county_id']

    HOST, dataset = "https://api.census.gov/data/" + str(year), "dec/sf1"
    get_vars =  ["P001" + str(i + 1).zfill(3) for i in range(1)]
    get_vars += ["P002" + str(i + 1).zfill(3) for i in range(5)]
    get_vars += ["P003" + str(i + 1).zfill(3) for i in range(8)]
    get_vars += ["P004" + str(i + 1).zfill(3) for i in range(3)]
    get_vars = ["NAME"] + get_vars

    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "county:*"
    predicates["in"] = "state:08"
    predicates["key"] = CENSUS_KEY
    #print(f'PREDICATES: {predicates}.')

    # Initialize data frame
    base_url = "/".join([HOST, dataset])
    r = requests.get(base_url, params=predicates)

    pop_df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    pop_df.columns = pop_col_names

    print(f'Population Dataframe shape: {pop_df.shape}')

    # Change Datatype for population variables to int64
    for i in range(1, pop_df.shape[1]):
        pop_df.iloc[:,i] = pop_df.iloc[:,i].astype(int)
    return(pop_df)
population_df = get_population_df(year = 2010)

population_df.describe()

Population Dataframe shape: (64, 20)


,total_pop,total_urb_rur,urban,urban_ins_ars,urban_ins_cls,rural,total_race,white,black,american_indian,asian,nat_hawaiian,some_othr_race,two_or_more_races,hisp_latino,not_hisp_latino,total_hisp_latino,state_id,county_id
count,64.00000,64.00000,64.000000,64.000000,64.00000,64.000000,64.00000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.0,64.000000
mean,78581.18750,78581.18750,67699.390625,60397.984375,7301.40625,10881.796875,78581.18750,63893.781250,3152.140625,875.156250,2172.31250,103.484375,5689.687500,2694.625000,78581.18750,62351.703125,16229.484375,8.0,62.234375
std,157027.62997,157027.62997,150562.413364,150552.140964,11475.32523,11537.590360,157027.62997,123363.297992,11450.748038,1695.723325,5666.67142,322.226185,13645.174846,6373.716076,157027.62997,124806.848755,37267.804962,0.0,36.880649
min,699.00000,699.00000,0.000000,0.000000,0.00000,0.000000,699.00000,647.000000,0.000000,2.000000,0.00000,0.000000,2.000000,10.000000,699.00000,615.000000,21.000000,8.0,1.000000
25%,5734.00000,5734.00000,0.000000,0.000000,0.00000,3967.250000,5734.00000,4985.000000,27.750000,55.500000,25.75000,2.750000,223.000000,107.000000,5734.00000,4317.250000,678.500000,8.0,30.500000
50%,15083.50000,15083.50000,5691.000000,0.000000,2763.00000,6642.000000,15083.50000,12694.500000,96.000000,154.000000,101.50000,9.000000,782.000000,329.500000,15083.50000,10585.500000,2237.500000,8.0,62.000000
75%,42663.00000,42663.00000,22574.000000,0.000000,9818.50000,13848.500000,42663.00000,37409.250000,511.750000,436.250000,279.00000,25.000000,2509.250000,910.750000,42663.00000,33988.750000,7231.500000,8.0,93.500000
max,622263.00000,622263.00000,600158.000000,600158.000000,55404.00000,55595.000000,622263.00000,496258.000000,61435.000000,8237.000000,29077.00000,2185.000000,71191.000000,31787.000000,622263.00000,528598.000000,190965.000000,8.0,125.000000


In [31]:
from datetime import date

def get_filename(name, census):
    fn = census + '-' + name + '.csv'
    return fn

sub_file = get_filename('population_df', 'Census2010')
OUTPUT_DATA_PATH = Path().absolute().parent / "data" / "processed" / sub_file
print(OUTPUT_DATA_PATH)

/home/jdreed/MSDS696/data/processed/Census2010-population_df.csv


In [33]:
population_df.to_csv(OUTPUT_DATA_PATH, index=True)

In [34]:
!head $OUTPUT_DATA_PATH

,county_state,total_pop,total_urb_rur,urban,urban_ins_ars,urban_ins_cls,rural,total_race,white,black,american_indian,asian,nat_hawaiian,some_othr_race,two_or_more_races,hisp_latino,not_hisp_latino,total_hisp_latino,state_id,county_id
0,"Costilla County, Colorado",3524,3524,0,0,0,3524,3524,2397,24,104,34,9,803,153,3524,1197,2327,8,23
1,"Crowley County, Colorado",5823,5823,0,0,0,5823,5823,4509,559,139,59,0,456,101,5823,4137,1686,8,25
2,"Custer County, Colorado",4255,4255,0,0,0,4255,4255,4055,44,30,16,0,39,71,4255,4055,200,8,27
3,"Delta County, Colorado",30952,30952,11399,0,11399,19553,30952,27761,160,295,160,15,1862,699,30952,26607,4345,8,29
4,"Denver County, Colorado",600158,600158,600158,600158,0,0,600158,413696,61435,8237,20433,607,71191,24559,600158,409193,190965,8,31
5,"Douglas County, Colorado",285465,285465,256098,247595,8503,29367,285465,257598,3476,1183,10716,192,4894,7406,285465,264073,21392,8,35
6,"Dolores County, Colorado",2064,2064,0,0,0,2064,2064,1920,4,60,2,2,26,50,2064,19

---
## Decennial Census - Gender

 - Housing Occupancy and Ownership (vacant./occupied, rent/own)
 - Group Quarters Population (prisons, college dorms)

### Build Gender DataFrame

In [22]:
def get_gender_df(year = 2010):

    '''
    Name
    P012001	Total SEX BY AGE
    P012002	Total!!Male	SEX BY AGE
    P012003	Total!!Male!!Under 5 years
    P012004	Total!!Male!!5 to 9 years
    P012005	Total!!Male!!10 to 14 years
    P012006	Total!!Male!!15 to 17 years
    P012007	Total!!Male!!18 and 19 years
    P012008	Total!!Male!!20 years
    P012009	Total!!Male!!21 years
    P012010	Total!!Male!!22 to 24 years
    P012011	Total!!Male!!25 to 29 years
    P012012	Total!!Male!!30 to 34 years
    P012013	Total!!Male!!35 to 39 years
    P012014	Total!!Male!!40 to 44 years
    P012015	Total!!Male!!45 to 49 years
    P012016	Total!!Male!!50 to 54 years
    P012017	Total!!Male!!55 to 59 years
    P012018	Total!!Male!!60 and 61 years
    P012019	Total!!Male!!62 to 64 years
    P012020	Total!!Male!!65 and 66 years
    P012021	Total!!Male!!67 to 69 years
    P012022	Total!!Male!!70 to 74 years
    P012023	Total!!Male!!75 to 79 years
    P012024	Total!!Male!!80 to 84 years
    P012025	Total!!Male!!85 years and over
    P012026	Total!!Female
    P012027	Total!!Female!!Under 5 years
    P012028	Total!!Female!!5 to 9 years
    P012029	Total!!Female!!10 to 14 years
    P012030	Total!!Female!!15 to 17 years
    P012031	Total!!Female!!18 and 19 years
    P012032	Total!!Female!!20 years
    P012033	Total!!Female!!21 years
    P012034	Total!!Female!!22 to 24 years
    P012035	Total!!Female!!25 to 29 years
    P012036	Total!!Female!!30 to 34 years
    P012037	Total!!Female!!35 to 39 years
    P012038	Total!!Female!!40 to 44 years
    P012039	Total!!Female!!45 to 49 years
    P012040	Total!!Female!!50 to 54 years
    P012041	Total!!Female!!55 to 59 years
    P012042	Total!!Female!!60 and 61 years
    P012043	Total!!Female!!62 to 64 years
    P012044	Total!!Female!!65 and 66 years
    P012045	Total!!Female!!67 to 69 years
    P012046	Total!!Female!!70 to 74 years
    P012047	Total!!Female!!75 to 79 years
    P012048	Total!!Female!!80 to 84 years
    P012049	Total!!Female!!85 years and over
    '''
    sex_col_names = ['county_state',    'total_sex_byage', 
                        'total_male',  'male_u5',     'male_5_9',      'male_10_14', 'male_15_17',   'male_18_19', 
                                       'male_20',     'male_21',       'male_22_24', 'male_25_29',   'male_30_34',   
                                       'male_35_39',  'male_40_44',    'male_45_49', 'male_50_54',   'male_55_59',   
                                       'male_60_61',  'male_62_64',    'male_65_66', 'male_67_69',   'male_70_74', 
                                       'male_75_79',  'male_80_84',    'male_85_over',

                        'total_female', 'female_u5',   'female_5_9',   'female_10_14', 'female_15_17', 'female_18_19', 
                                        'female_20',   'female_21',    'female_22_24', 'female_25_29', 'female_30_34', 
                                        'female_35_39','female_40_44', 'female_45_49', 'female_50_54', 'female_55_59', 
                                        'female_60_61','female_62_64', 'female_65_66', 'female_67_69', 'female_70_74',
                                        'female_75_79','female_80_84', 'female_85_over',
                      'state_id', 'county_id' ]

    HOST, dataset = "https://api.census.gov/data/" + str(year), "dec/sf1"
    get_vars =  ["P012" + str(i + 1).zfill(3) for i in range(49)]
    get_vars = ["NAME"] + get_vars
    #print(f'GET_VARS: {get_vars}')
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "county:*"
    predicates["in"] = "state:08"
    predicates["key"] = CENSUS_KEY

    # Initialize data
    base_url = "/".join([HOST, dataset])
    r = requests.get(base_url, params=predicates)

    gender_df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    gender_df.columns = sex_col_names

    print(f'Gender Dataframe shape: {gender_df.shape}')

    # Change Datatype for population variables to int64
    for i in range(1, gender_df.shape[1]):
        gender_df.iloc[:,i] = gender_df.iloc[:,i].astype(int)

    return(gender_df)

gender_df = get_gender_df(year = 2010)

gender_df.describe()

Gender Dataframe shape: (64, 52)


,total_sex_byage,total_male,male_u5,male_5_9,male_10_14,male_15_17,male_18_19,male_20,male_21,male_22_24,...,female_60_61,female_62_64,female_65_66,female_67_69,female_70_74,female_75_79,female_80_84,female_85_over,state_id,county_id
count,64.00000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,...,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.0,64.000000
mean,78581.18750,39385.343750,2755.640625,2777.843750,2659.250000,1611.56250,1134.218750,566.312500,550.906250,1722.859375,...,920.140625,1218.781250,644.562500,836.031250,1057.734375,835.187500,677.671875,726.484375,8.0,62.234375
std,157027.62997,77999.350262,5731.162580,5661.154448,5395.236132,3248.15466,2298.264297,1190.993435,1167.530642,3633.456673,...,1780.523624,2352.923183,1229.267311,1594.255899,2051.201632,1642.779525,1356.967453,1463.934880,0.0,36.880649
min,699.00000,362.000000,8.000000,22.000000,14.000000,8.00000,2.000000,2.000000,1.000000,9.000000,...,12.000000,20.000000,5.000000,12.000000,6.000000,5.000000,2.000000,3.000000,8.0,1.000000
25%,5734.00000,3152.500000,144.750000,148.750000,154.500000,100.50000,48.750000,24.750000,25.750000,90.000000,...,78.000000,103.000000,60.000000,71.750000,101.000000,75.000000,52.500000,50.500000,8.0,30.500000
50%,15083.50000,7837.000000,417.000000,426.500000,418.000000,292.00000,173.500000,81.000000,79.000000,254.000000,...,198.500000,255.500000,137.500000,182.000000,205.000000,145.000000,110.500000,115.000000,8.0,62.000000
75%,42663.00000,21767.000000,1144.000000,1213.500000,1268.000000,851.75000,477.250000,269.250000,271.000000,996.000000,...,527.250000,723.250000,361.250000,454.250000,590.000000,446.250000,377.250000,428.750000,8.0,93.500000
max,622263.00000,310162.000000,23152.000000,23050.000000,23252.000000,14097.00000,10417.000000,5512.000000,5363.000000,15708.000000,...,7215.000000,9738.000000,5027.000000,6641.000000,8375.000000,6725.000000,5905.000000,6513.000000,8.0,125.000000


In [35]:
sub_file = get_filename('gender_df','Census2010')
OUTPUT_DATA_PATH = Path().absolute().parent / "data" / "processed" / sub_file
print(OUTPUT_DATA_PATH)

/home/jdreed/MSDS696/data/processed/Census2010-gender_df.csv


In [36]:
gender_df.to_csv(OUTPUT_DATA_PATH, index=True)

In [37]:
!head $OUTPUT_DATA_PATH

,county_state,total_sex_byage,total_male,male_u5,male_5_9,male_10_14,male_15_17,male_18_19,male_20,male_21,male_22_24,male_25_29,male_30_34,male_35_39,male_40_44,male_45_49,male_50_54,male_55_59,male_60_61,male_62_64,male_65_66,male_67_69,male_70_74,male_75_79,male_80_84,male_85_over,total_female,female_u5,female_5_9,female_10_14,female_15_17,female_18_19,female_20,female_21,female_22_24,female_25_29,female_30_34,female_35_39,female_40_44,female_45_49,female_50_54,female_55_59,female_60_61,female_62_64,female_65_66,female_67_69,female_70_74,female_75_79,female_80_84,female_85_over,state_id,county_id
0,"Costilla County, Colorado",3524,1822,99,102,107,95,41,15,19,32,71,73,72,96,130,149,151,57,110,74,89,109,69,44,18,1702,76,94,98,65,42,16,12,44,69,71,77,87,113,150,146,54,84,62,72,102,66,51,51,8,23
1,"Crowley County, Colorado",5823,4199,88,110,136,81,46,59,65,283,546,479,430,401,438,327,245,72,96,50,67,83,40,31,26,1624,94,100,104,76,47,13,11,40,69,83,80,89,123,128,137,37,76,37,62,52,61,54,

---
## Decennial Census 2010

Housing survey
 - Housing Occupancy and Ownership (vacant/occupied, rent/own)
 - Group Quarters Population (prisons, college dorms)

### Build Housing DataFrame

In [23]:
def get_housing_df(year = 2010):

    '''
    Name
    P018001	Total
    P018002	Total!!Family households
    P018003	Total!!Family households!!Husband-wife family
    P018004	Total!!Family households!!Other family
    P018005	Total!!Family households!!Other family!!Male householder, no wife present
    P018006	Total!!Family households!!Other family!!Female householder, no husband present
    P018007	Total!!Nonfamily households
    P018008	Total!!Nonfamily households!!Householder living alone
    P018009	Total!!Nonfamily households!!Householder not living alone
    H017001	Total	TENURE BY AGE OF HOUSEHOLDER
    H017002	Total!!Owner occupied
    H017003	Total!!Owner occupied!!Householder 15 to 24 years
    H017004	Total!!Owner occupied!!Householder 25 to 34 years
    H017005	Total!!Owner occupied!!Householder 35 to 44 years
    H017006	Total!!Owner occupied!!Householder 45 to 54 years
    H017007	Total!!Owner occupied!!Householder 55 to 59 years
    H017008	Total!!Owner occupied!!Householder 60 to 64 years
    H017009	Total!!Owner occupied!!Householder 65 to 74 years
    H017010	Total!!Owner occupied!!Householder 75 to 84 years
    H017011	Total!!Owner occupied!!Householder 85 years and over
    H017012	Total!!Renter occupied	TENURE BY AGE OF HOUSEHOLDER
    H017013	Total!!Renter occupied!!Householder 15 to 24 years
    H017014	Total!!Renter occupied!!Householder 25 to 34 years
    H017015	Total!!Renter occupied!!Householder 35 to 44 years
    H017016	Total!!Renter occupied!!Householder 45 to 54 years
    H017017	Total!!Renter occupied!!Householder 55 to 59 years
    H017018	Total!!Renter occupied!!Householder 60 to 64 years
    H017019	Total!!Renter occupied!!Householder 65 to 74 years
    H017020	Total!!Renter occupied!!Householder 75 to 84 years
    H017021	Total!!Renter occupied!!Householder 85 years and over
    '''
                          # 1
    housing_col_names = ['county_state',  

             'total_house_tenure', 'total_occupied',# 10
             'households_fam',    'households_husband_wife',  'households_other', 
             'households_other_male', 'households_other_owner', 'households_nonfam',  
             'householder_living_alone', 'householder_not_living_alone', 

             'total_owner', 'owner_15_24', 'owner_25_34', 'owner_35_44', 'owner_45_54',   
             'owner_55_59', 'owner_60_64', 'owner_65_74', 'owner_75_84', 'owner_85_up',

             'total_renter', 'renter_15_24', 'renter_25_34', 'renter_35_44', 'renter_45_54',   
             'renter_55_59',  'renter_60_64', 'renter_65_74', 'renter_75_84', 'renter_85_up',

          'state_id', 'county_id' ]

    HOST, dataset = "https://api.census.gov/data/" + str(year), "dec/sf1"
    get_vars  =  ["P018" + str(i + 1).zfill(3) for i in range(9)]
    get_vars +=  ["H017" + str(i + 1).zfill(3) for i in range(21)]
    get_vars  = ["NAME"] + get_vars
    #print(f'GET_VARS: {get_vars}')
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "county:*"
    predicates["in"] = "state:08"
    predicates["key"] = CENSUS_KEY

    # Initialize data frame
    base_url = "/".join([HOST, dataset])
    r = requests.get(base_url, params=predicates)

    housing_df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    #print(f'housing_df features: {housing_df.columns}')
    housing_df.columns = housing_col_names

    print(f'Housing Dataframe shape: {housing_df.shape}')

    # Change Datatype for population variables to int64
    for i in range(1, housing_df.shape[1]):
        housing_df.iloc[:,i] = housing_df.iloc[:,i].astype(int)
    return(housing_df)

housing_df = get_housing_df(2010)
housing_df.describe()

Housing Dataframe shape: (64, 33)


,total_house_tenure,total_occupied,households_fam,households_husband_wife,households_other,households_other_male,households_other_owner,households_nonfam,householder_living_alone,householder_not_living_alone,...,renter_25_34,renter_35_44,renter_45_54,renter_55_59,renter_60_64,renter_65_74,renter_75_84,renter_85_up,state_id,county_id
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,...,64.000000,64.000000,64.00000,64.000000,64.00000,64.000000,64.000000,64.000000,64.0,64.000000
mean,30826.062500,19711.359375,15178.078125,4533.281250,1426.546875,3106.734375,11114.703125,8606.156250,2508.546875,30826.062500,...,3026.140625,2050.093750,1729.00000,647.890625,488.31250,542.625000,377.750000,271.875000,8.0,62.234375
std,62189.745293,38956.548991,29509.677221,9690.349977,2931.714656,6768.228807,24461.697644,19023.578647,5545.149769,62189.745293,...,7429.250728,4627.203083,3732.24226,1409.021839,1074.31422,1195.469523,813.072499,588.596569,0.0,36.880649
min,344.000000,156.000000,123.000000,24.000000,10.000000,11.000000,123.000000,101.000000,13.000000,344.000000,...,8.000000,14.000000,13.00000,7.000000,5.00000,3.000000,1.000000,0.000000,8.0,1.000000
25%,2011.500000,1343.250000,1172.500000,268.500000,91.250000,171.000000,725.250000,645.750000,85.500000,2011.500000,...,124.250000,116.500000,118.75000,50.750000,40.25000,42.750000,27.000000,15.000000,8.0,30.500000
50%,6189.500000,3756.000000,2908.500000,741.500000,257.500000,459.000000,1979.500000,1632.000000,347.500000,6189.500000,...,353.000000,313.000000,310.00000,135.000000,95.00000,99.500000,56.000000,32.000000,8.0,62.000000
75%,16508.500000,11074.000000,8755.750000,2088.750000,727.500000,1301.000000,5310.000000,4259.250000,1340.750000,16508.500000,...,1114.000000,854.250000,772.50000,305.000000,233.50000,288.750000,210.500000,148.000000,8.0,93.500000
max,263107.000000,160489.000000,123304.000000,39257.000000,11435.000000,27822.000000,138103.000000,106828.000000,31275.000000,263107.000000,...,44471.000000,23896.000000,18148.00000,7647.000000,6092.00000,6962.000000,4480.000000,2908.000000,8.0,125.000000


In [39]:
sub_file = get_filename('housing_df', 'Census2010')
OUTPUT_DATA_PATH = Path().absolute().parent / "data" / "processed" / sub_file
print(OUTPUT_DATA_PATH)

/home/jdreed/MSDS696/data/processed/Census2010-housing_df.csv


In [40]:
housing_df.to_csv(OUTPUT_DATA_PATH, index=True)

In [41]:
!head $OUTPUT_DATA_PATH

,county_state,total_house_tenure,total_occupied,households_fam,households_husband_wife,households_other,households_other_male,households_other_owner,households_nonfam,householder_living_alone,householder_not_living_alone,total_owner,owner_15_24,owner_25_34,owner_35_44,owner_45_54,owner_55_59,owner_60_64,owner_65_74,owner_75_84,owner_85_up,total_renter,renter_15_24,renter_25_34,renter_35_44,renter_45_54,renter_55_59,renter_60_64,renter_65_74,renter_75_84,renter_85_up,state_id,county_id
0,"Costilla County, Colorado",1550,976,691,285,105,180,574,520,54,1550,1195,10,44,125,229,156,164,276,146,45,355,26,65,47,65,31,30,56,26,9,8,23
1,"Crowley County, Colorado",1306,856,646,210,71,139,450,394,56,1306,930,9,67,88,207,122,112,174,109,42,376,31,65,71,68,44,32,36,16,13,8,25
2,"Custer County, Colorado",1925,1272,1135,137,61,76,653,580,73,1925,1562,8,51,131,314,247,260,364,164,23,363,13,54,64,78,39,41,36,22,16,8,27
3,"Delta County, Colorado",12703,8652,7137,1515,510,1005,4051,3486,565,12703,9569,14

## American Community Survey
 - Educational Attainment
 - Commuting (mode, time leaving, time travelled)
 - Disability Status

In [77]:
def get_acs19_pop(year = 2019):

    '''
    Name
    Supplemental Estimates:
    API Example Call: api.census.gov/data/2019/acs/acsse?get=NAME,K200101_001E&for=state:*&key=YOUR_KEY_GOES_HERE
    K200101_001E	Estimate!!Total:	POPULATION BY SEX	 
    K200101_002E	Estimate!!Total:!!Male	POPULATION BY SEX	 
    K200101_003E	Estimate!!Total:!!Female	POPULATION BY SEX	 
    K200102_001E	Estimate!!Total:	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_002E	Estimate!!Total:!!In households:	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_003E	Estimate!!Total:!!In households:!!Under 3 years	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_004E	Estimate!!Total:!!In households:!!3 to 5 years	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_005E	Estimate!!Total:!!In households:!!6 to 8 years	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_006E	Estimate!!Total:!!In households:!!9 to 11 years	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_007E	Estimate!!Total:!!In households:!!12 to 14 years	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_008E	Estimate!!Total:!!In households:!!15 to 17 years	POPULATION UNDER 18 YEARS BY AGE	 
    K200102_009E	Estimate!!Total:!!In group quarters	POPULATION UNDER 18 YEARS BY AGE	 
    K200103_001E	Estimate!!Median age --!!Total:	MEDIAN AGE BY SEX	 
    K200103_002E	Estimate!!Median age --!!Male	MEDIAN AGE BY SEX	 
    K200103_003E	Estimate!!Median age --!!Female	MEDIAN AGE BY SEX	 
    K200104_001E	Estimate!!Total:	POPULATION BY AGE	 
    K200104_002E	Estimate!!Total:!!Under 18 years	POPULATION BY AGE	 
    K200104_003E	Estimate!!Total:!!18 to 24 years	POPULATION BY AGE	 
    K200104_004E	Estimate!!Total:!!25 to 34 years	POPULATION BY AGE	 
    K200104_005E	Estimate!!Total:!!35 to 44 years	POPULATION BY AGE	 
    K200104_006E	Estimate!!Total:!!45 to 54 years	POPULATION BY AGE	 
    K200104_007E	Estimate!!Total:!!55 to 64 years	POPULATION BY AGE	 
    K200104_008E	Estimate!!Total:!!65 years and over	POPULATION BY AGE	 
    '''
    acs_pop_col_names = ['county_state',  
              'pop_by_sex', 'pbs_male', 'pbs_female' , 'pop_und18',
              'pop_und18_by_age', 'pop_lt3', 'pop_3_5', 'pop_6_8',
              'pop_9_11', 'pop_12_14', 'pop_15_17', 'pop_gq_und_18',
              'median_age', 'median_age_male', 'median_age_female',
              'total_pop_by_age', 'pop_all_und18', 'pop_all_18_24',
              'pop_all_25_34', 'pop_all_35_44', 'pop_all_45_54',
              'pop_all_55_64', 'pop_all_65_over',
              'state_id', 'county_id' ]

    HOST, dataset = "https://api.census.gov/data/" + str(year), "acs/acsse"
    get_vars   =  ["K200101_" + str(i + 1).zfill(3) + "E" for i in range(3)]
    get_vars  +=  ["K200102_" + str(i + 1).zfill(3) + "E" for i in range(9)]
    get_vars  +=  ["K200103_" + str(i + 1).zfill(3) + "E" for i in range(3)]
    get_vars  +=  ["K200104_" + str(i + 1).zfill(3) + "E" for i in range(8)]
    get_vars   = ["NAME"] + get_vars
    #print(f'GET_VARS: {get_vars}')
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "county:*"
    predicates["in"] = "state:08"
    predicates["key"] = CENSUS_KEY

    # Initialize data frame
    base_url = "/".join([HOST, dataset])
    r = requests.get(base_url, params=predicates)

    acspop_df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    acspop_df.columns = acs_pop_col_names

    print(f'ACS Dataframe shape: {acspop_df.shape}')

    # Change Datatype for population variables to int64
    acspop_df.fillna(0, inplace = True)
    acspop_df = acs19_pop_df.astype({'pop_by_sex': int, 'pbs_male': int, 'pbs_female': int, 
                'pop_und18': int, 'pop_und18_by_age': int, 'pop_lt3': int, 'pop_3_5': int, 
                'pop_6_8': int, 'pop_9_11': int, 'pop_12_14': int, 'pop_15_17': int, 
                'pop_gq_und_18': int, 'total_pop_by_age': int, 'pop_all_und18': int, 
                'pop_all_18_24': int, 'pop_all_25_34': int, 'pop_all_35_44': int, 
                'pop_all_45_54': int, 'pop_all_55_64': int, 'pop_all_65_over': int, 
                'state_id': int, 'county_id':int, 'median_age': float, 'median_age_male': float, 
                'median_age_female': float})
    return(acspop_df)

acs19_pop_df = get_acs19_pop(2019)
acs19_pop_df.describe()
acs19_pop_df.dtypes

ACS Dataframe shape: (27, 26)


county_state          object
pop_by_sex             int64
pbs_male               int64
pbs_female             int64
pop_und18              int64
pop_und18_by_age       int64
pop_lt3                int64
pop_3_5                int64
pop_6_8                int64
pop_9_11               int64
pop_12_14              int64
pop_15_17              int64
pop_gq_und_18          int64
median_age           float64
median_age_male      float64
median_age_female    float64
total_pop_by_age       int64
pop_all_und18          int64
pop_all_18_24          int64
pop_all_25_34          int64
pop_all_35_44          int64
pop_all_45_54          int64
pop_all_55_64          int64
pop_all_65_over        int64
state_id               int64
county_id              int64
dtype: object

In [ ]:
sub_file = get_filename('acs19_pop_df', 'ACS19')
OUTPUT_DATA_PATH = Path().absolute().parent / "data" / "processed" / sub_file
print(OUTPUT_DATA_PATH)

In [40]:
acs19_pop_df.to_csv(OUTPUT_DATA_PATH, index=True)

## American Community Survey
 - Marital Status: 
     + Never married
     + Now married
     + Separated
     + Widowed
     + Divorced

In [82]:
def get_acs19_marital(year = 2019):

    '''
    Name
    Supplemental Estimates:
    API Example Call: api.census.gov/data/2019/acs/acsse?get=NAME,K200101_001E&for=state:*&key=YOUR_KEY_GOES_HERE
    K201001_001E	Estimate!!Total:	MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER	 
    K201001_002E	Estimate!!Total:!!Never married	MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER	 
    K201001_003E	Estimate!!Total:!!Now married (except separated)	MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER	 
    K201001_004E	Estimate!!Total:!!Separated	MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER	 
    K201001_005E	Estimate!!Total:!!Widowed	MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER	 
    K201001_006E	Estimate!!Total:!!Divorced	MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER	 
    '''
    acs_marital_col_names = ['county_state',  
              'total_marital', 'never_married', 'now_married', 'separated',
              'widowed', 'divorced','state_id', 'county_id' ]

    HOST, dataset = "https://api.census.gov/data/" + str(year), "acs/acsse"
    get_vars   =  ["K201001_" + str(i + 1).zfill(3) + "E" for i in range(6)]
    get_vars   = ["NAME"] + get_vars
    #print(f'GET_VARS: {get_vars}')
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "county:*"
    predicates["in"] = "state:08"
    predicates["key"] = CENSUS_KEY

    # Initialize data frame
    base_url = "/".join([HOST, dataset])
    r = requests.get(base_url, params=predicates)

    acsmar_df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    acsmar_df.columns = acs_marital_col_names

    print(f'ACS Dataframe shape: {acsmar_df.shape}')

    # Change Datatype for population variables to int64
    acsmar_df.fillna(0, inplace = True)
    acsmar_df = acsmar_df.astype({'total_marital': int, 'never_married': int, 'now_married': int, 'separated': int,
              'widowed': int, 'divorced': int,'state_id': int, 'county_id': int })
    return(acsmar_df)

acs19_mar_df = get_acs19_marital(2019)
acs19_mar_df.describe()
acs19_mar_df.dtypes
acs19_mar_df

ACS Dataframe shape: (27, 9)


,county_state,total_marital,never_married,now_married,separated,widowed,divorced,state_id,county_id
0,"Chaffee County, Colorado",18661,3303,11236,47,1513,2562,8,15
1,"Garfield County, Colorado",47774,13264,27474,500,1713,4823,8,45
2,"Jefferson County, Colorado",490331,147548,256343,5533,21255,59652,8,59
3,"Adams County, Colorado",405343,132990,201572,8200,14712,47869,8,1
4,"Eagle County, Colorado",45961,17827,22549,202,484,4899,8,37
5,"Montezuma County, Colorado",22217,3599,13536,807,893,3382,8,83
6,"Boulder County, Colorado",276608,106304,127359,2302,10691,29952,8,13
7,"Routt County, Colorado",21186,6884,10356,193,663,3090,8,107
8,"Pueblo County, Colorado",137908,43952,63598,2168,7716,20474,8,101
9,"Mesa County, Colorado",127048,31346,72477,1377,5547,16301,8,77


In [79]:
sub_file = get_filename('acs19_mar_df', 'ACS19')
OUTPUT_DATA_PATH = Path().absolute().parent / "data" / "processed" / sub_file
print(OUTPUT_DATA_PATH)

/home/jdreed/MSDS696/data/processed/ACS19-acs19_mar_df.csv


In [80]:
acs19_mar_df.to_csv(OUTPUT_DATA_PATH, index=True)

## Colorado State Census Data

### Annual Estimates of the Resident Population 2010 - 2019

---

## Colorado State Voting Records

### Election Results

**2018**, **2016**, **2014**, **2012**, **2010**  
    - General Election Results  
    - General Election precinct-level turnout results  


### Voter Registration by County and Party Affiliation

**2019**, **2018**, **2016**, **2014**, **2012**, **2010** **2018**, **2016**, **2014**, **2012**, and **2010** 

---

Structure of a Subject Table

In [ ]:
HOST, dataset = "https://api.census.gov/data", "acs/acs1"
get_vars = ["B25045_" + str(i + 1).zfill(3) + "E" for i in range(19)]
get_vars = ["NAME"] + get_vars
print(get_vars)
predicates = {}
predicates["get"] = ",".join(get_vars)
predicates["for"] = "us:*"
predicates["key"] = CENSUS_KEY
print(predicates)

In [ ]:
# Initialize data frame collector
dfs = []
responses = []
for year in range(2011, 2018):
    base_url = "/".join([HOST, str(year), dataset])
    r = requests.get(base_url, params=predicates)
    print(f'base_url: {base_url}')
    df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    # Add column to hold year value
    df["year"] = year
    dfs.append(df)
# Concatenate all data frames in collector
us = pd.concat(dfs)

In [ ]:
print(predicates)
print(r.url)

In [ ]:
import os
